### Problem 1

a) Code the Conjugate Gradient Method and apply it to solve a simple $$Hx=b$$ system, with H’s dimension as 10×10.

In [17]:
# r_k = Hx - b
from this import d
import numpy as np

epsilon_threshhold = 0.001

def conjugate_gradient(H,b,initial_point = np.array([[3],[1]])):

    x_k = initial_point
    r_k = np.matmul(H,initial_point) - b
    p_k = -r_k

    while np.linalg.norm(r_k) > epsilon_threshhold:
        
        a_k_num = -np.matmul(r_k.T,p_k)
        a_k_denom = np.matmul(p_k.T,np.matmul(H,p_k))
        a_k = a_k_num / a_k_denom


        x_k = x_k + a_k*p_k
        r_k = np.matmul(H,x_k) - b



        b_k_num = np.matmul(p_k.T,np.matmul(H,r_k))
        b_k_denom = np.matmul(p_k.T,np.matmul(H,p_k))

        b_k = b_k_num / b_k_denom

        p_k = -r_k + b_k*p_k


    print(x_k)



#H = np.array([[8,-2],[-2,2]])
#initial_point = np.array([[3],[1]])
#b = np.array([[1],[1]])

H = np.array([[ 2.5409,-0.0113],
  [-0.0113,0.5287]])
b = np.array([[1.3864], [0.3719]])
initial_point  = np.array([[-3],[-4]])

conjugate_gradient(H,b,initial_point)


[[0.5488139 ]
 [0.71515339]]


In [11]:
y = np.matmul(b.T,np.matmul(H,b))

In [12]:
y

array([[6]])

array([[0.5],
       [0.5]])